In [ ]:
! pip install transformers==4.5.0
! pip install spacy==2.0.12
! pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.0.12-cp37-cp37m-linux_x86_64.whl
  Using cached regex-2017.4.5-cp37-cp37m-linux_x86_64.whl
  Using cached thinc-6.10.3.tar.gz (1.2 MB)
  Using cached preshed-1.0.1-cp37-cp37m-manylinux1_x86_64.whl (79 kB)
  Using cached cymem-1.31.2-cp37-cp37m-linux_x86_64.whl
  ERROR: Failed building wheel for thinc
  Running setup.py clean for thinc
Failed to build thinc
  Attempting uninstall: cymem
    Found existing installation: cymem 2.0.6
    Uninstalling cymem-2.0.6:
      Successfully uninstalled cymem-2.0.6
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Succe

In [ ]:
!pip install tensorflow==2.0.0
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn
!pip3 install sentencepiece
!pip3 install tf_sentencepiece

In [ ]:
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import re

import glob
#stop words
from nltk.corpus import stopwords
#tokenize
from nltk.tokenize import word_tokenize

from tqdm.auto import tqdm
from drive.MyDrive.pyrouge import Rouge

tqdm.pandas()

In [ ]:
#Stop word removal function
import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

#from spacy.lang.en.stop_words import STOP_WORDS

def stop_word_remove(sentence):
    temp = [token for token in sentence.split() if token not in all_stopwords]
    return ' '.join(word for word in temp)

In [ ]:
path = '/content/drive/MyDrive/Parsed_Papers/Introduction/Introduction/S0167739X13001349.txt'

In [ ]:
def read_paper(path):
  f = open(path, 'r')
  text = str(f.read().strip())
  return text

In [ ]:
t = read_paper(path)

In [ ]:
def bodyMain(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = (text.encode('ascii','ignore')).decode("utf-8")

  #Extract body from the paper

  body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text, flags= re.I)[0]

  # body = ' '.join(body_main.split())
  # body = body.split(".")
  return body_main

In [ ]:
x = bodyMain(t)
x

'   Highly-threaded, many-core devices such as GPUs have gained popularity in the last decade; both NVIDIA and AMD manufacture general purpose GPUs that fall in this category. The important distinction between these machines and traditional multi-core machines is that these devices provide a large number of low-overhead hardware threads with low-overhead context switching between them; this fast context-switch mechanism is used to hide the memory access latency of transferring data from slow large (and often global) memory to fast, small (and typically local) memory. Researchers have designed algorithms to solve many interesting problems for these devices, such as GPU sorting or hashing [14], linear algebra [57], dynamic programming [8,9], graph algorithms [1013], and many other classic algorithms [14,15]. These projects generally report impressive gains in performance. These devices appear to be here to stay. While there is a lot of folk wisdom on how to design good algorithms for the

In [ ]:
def extract(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = (text.encode('ascii','ignore')).decode("utf-8")

  #Extract highlights, body from body of the paper 
  highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHASES', text, flags = re.I)[0]
  body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text, flags= re.I)[0]

In [ ]:
def abstract(text):
  text = re.sub("@&#", " ", text)
  text = re.sub("\n", " ", text)
  text = ' '.join(text.split())
  text = (text.encode('ascii','ignore')).decode("utf-8")

  abst = re.findall(r'ABSTRACT(.*?)INTRODUCTION', text, flags = re.I)[0]
  # print("abs_main: ",abst)
  return abst

In [ ]:
def process_paper(text):
  # Removes unwanted characters, accounting for unicode characters
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = ' '.join(text.split())
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    

  # Extracting the highlights, body from the paper
    highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
    abst = re.findall(r'.*(?:abstract)(.*?)introduction', text, flags=re.I)[0]
    body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text,  flags = re.I)[0]
    #body_main = re.findall(r'.*(?:abstract)(.*?)references', text, flags=re.I)[0]
    
    # Making a copy of the body, lowercasing body text, removing punctuations & extra spaces
    dummy_body = body_main.lower()
    dummy_body = re.sub('[^\w\s\d\.]','',dummy_body)
    dummy_body = ' '.join(dummy_body.split())
    dummy_body = dummy_body.split(".")

  # Removing extra spaces from the body text, which will be preserved to produce summaries
  # And splitting into sentences
    body = ' '.join(body_main.split())
    body = body.split(".")

  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_body):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_body.pop(i)
            body.pop(i)
            
    # Making a copy of the abstract, lowercasing abstract text, removing punctuations & extra spaces
    dummy_abst = abst.lower()
    dummy_abst = re.sub('[^\w\s\d\.]','',dummy_abst)
    dummy_abst = ' '.join(dummy_abst.split())
    dummy_abst = dummy_abst.split(".")
    
  # Removing extra spaces from the abstract text, which will be preserved to produce summaries
  # And splitting into sentences
    ab = ' '.join(abst.split())
    ab = abst.split(".")
    
  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_abst):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_abst.pop(i)
            ab.pop(i)

  # Making a copy of the highlights, lowercasing body text, removing punctuations & extra spaces
    dummy_highlights = highlights.lower()
    dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights)
    dummy_highlights = ' '.join(dummy_highlights.split())

  # Removing stop words from body & highlights
    body_copy = []
    for x in dummy_body:
      
        body_copy.append(stop_word_remove(x))
    highlight_copy = []
    for x in dummy_highlights.split():
        highlight_copy.append(stop_word_remove(x))
        
    # Removing stop words from abstarct
    abst_copy = []
    for x in dummy_abst:
      
        abst_copy.append(stop_word_remove(x))
        
    # Combing all of the abstract's sentences into one string    
    abst_copy = " ".join(sentence for sentence in abst_copy)
    abst_copy = " ".join(abst_copy.split())

  
  # Combing all of the highlights into one string    
    highlight_copy = " ".join(sentence for sentence in highlight_copy)
    highlight_copy = " ".join(highlight_copy.split())
    #print("BC",body_copy)
    
    return body_main, body_copy, highlights, highlight_copy, abst , abst_copy


In [ ]:
body_main, body_copy, highlights, highlight_copy, abst , abst_copy = process_paper(t)

In [ ]:
body_main

' Highly-threaded, many-core devices such as GPUs have gained popularity in the last decade; both NVIDIA and AMD manufacture general purpose GPUs that fall in this category. The important distinction between these machines and traditional multi-core machines is that these devices provide a large number of low-overhead hardware threads with low-overhead context switching between them; this fast context-switch mechanism is used to hide the memory access latency of transferring data from slow large (and often global) memory to fast, small (and typically local) memory. Researchers have designed algorithms to solve many interesting problems for these devices, such as GPU sorting or hashing [14], linear algebra [57], dynamic programming [8,9], graph algorithms [1013], and many other classic algorithms [14,15]. These projects generally report impressive gains in performance. These devices appear to be here to stay. While there is a lot of folk wisdom on how to design good algorithms for these

In [ ]:
body_copy

['important distinction machines traditional multicore machines devices provide large number lowoverhead hardware threads lowoverhead context switching fast contextswitch mechanism hide memory access latency transferring data slow large global memory fast small typically local memory',
 'projects generally report impressive gains performance',
 'devices appear stay',
 'interested analyzing characterizing performance algorithms highlythreaded manycore machines abstract algorithmic systematic manner',
 'years scientists designed models capture important aspects machines use',
 'model assumes operations including memory accesses unit time',
 'aggarwal vitter proposed disk access machine dam model 22 counts number memory transfers slow fast memory instead simply counting number memory accesses program',
 'number models consider memory access costs sequential algorithms different ways 2329',
 'pram model algorithms complexity analyzed terms workthe time taken algorithm 1 processor span call

In [ ]:
highlights

' We design a memory model to analyze algorithms for highly-threaded many-core systems. The model captures significant factors of performance: work, span, and memory accesses. We show the model is better than PRAM by applying both to 4 shortest paths algorithms. Empirical performance is effectively predicted by our model in many circumstances. It is the first formalized asymptotic model helpful for algorithm design on many-cores. '

In [ ]:
highlight_copy

'design memory model analyze algorithms highlythreaded manycore systems model captures significant factors performance work span memory accesses model better pram applying 4 shortest paths algorithms empirical performance effectively predicted model circumstances formalized asymptotic model helpful algorithm design manycores'

In [ ]:
abst

' A number of highly-threaded, many-core architectures hide memory-access latency by low-overhead context switching among a large number of threads. The speedup of a program on these machines depends on how well the latency is hidden. If the number of threads were infinite, theoretically, these machines could provide the performance predicted by the PRAM analysis of these programs. However, the number of threads per processor is not infinite, and is constrained by both hardware and algorithmic limits. In this paper, we introduce the Threaded Many-core Memory (TMM) model which is meant to capture the important characteristics of these highly-threaded, many-core machines. Since we model some important machine parameters of these machines, we expect analysis under this model to provide a more fine-grained and accurate performance prediction than the PRAM analysis. We analyze 4 algorithms for the classic all pairs shortest paths problem under this model. We find that even when two algorith

In [ ]:
abst_copy

'speedup program machines depends latency hidden number threads processor infinite constrained hardware algorithmic limits model important machine parameters machines expect analysis model provide finegrained accurate performance prediction pram analysis analyze 4 algorithms classic pairs shortest paths problem model example dense graphs dynamic programming algorithm johnsons algorithm performance pram model validate predictions model empirical measurements instantiation highlythreaded manycore machine nvidia gtx 480'

# **Summary**

In [ ]:
test = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/*.txt")

In [ ]:
len(test)

40

In [ ]:
col_names = ['FileName','Abstract','Summary','Precision','Recall','F1_Score']

result = pd.DataFrame(columns=col_names,dtype=object)

for i,path in enumerate(tqdm(test[:])):

  file_name = path.split('/')[-1].replace('.txt', '')
  text = read_paper(path)
  abstractL = abstract(text)
  body_main = bodyMain(text)

  model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
  
  summary = ''.join(model(body_main, min_length=60))
  
  r = Rouge()

  [precision, recall, f1_score] = r.rouge_l([abstractL], [summary])

  df = pd.DataFrame(columns=col_names,dtype=object)

  df.loc[i, ['FileName']] = file_name
  df.loc[i,['Abstract']] = abstractL
  df.loc[i,['Summary']] =  summary
  df.loc[i, ['Precision']] = precision
  df.loc[i, ['Recall']] = recall
  df.loc[i, ['F1_Score']] = f1_score

  result = pd.concat([result, df], axis = 0)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bi

In [ ]:
result.head(2)

,FileName,Abstract,Summary,Precision,Recall,F_Score,F1_Score
0,S0168874X14001413,The topic of the paper is related to the macr...,Finite element analysis (FEA) of airframe unde...,0.699306,0.173352,NaN,0.277832
1,S0168874X14001425,Multi-scale finite element (FE) modeling and ...,Finite element (FE) modeling and analysis of a...,0.833016,0.172905,NaN,0.286370


In [ ]:
result.to_csv('/content/drive/MyDrive/project/result/GPT2_Extractive_Text_Summarization.csv')